**Plot aerocet data with D3 JS scrip**

1. Based on [this](http://roadtolarissa.com/meteor-map/) and its source [code](https://github.com/1wheel/meteors/blob/master/metors.json).
1. The meteor web application is based on one index.html file and marker visualizer based on  JS drawMap.js. 
1. The index.html was edited for to remove the CHART in the line 14 to 19 contains 
```html
<div id="year-chart" class="chart display">
			<div class="title">Year of Impact</div>
		</div>

		<div id="mass-chart" class="chart display">
			<div class="title">Mass (g)</div>
		</div>
```
1. IN drawMap.js, following lines were removed
```javascript
function renderAll(){
		chart.each(render);
		var filterArray = cartoDbIds.all();
		filterArray.forEach(function(d, i){
			if (d.value != lastFilterArray[i]){
				lastFilterArray[i] = d.value;
				d3.select("#id" + d.key).transition().duration(500)
						.attr("r", d.value == 1 ? 2*metorScale(metors[i].mass) : 0)
					.transition().delay(550).duration(500)
						.attr("r", d.value == 1 ? metorScale(metors[i].mass) : 0);
			}
		})
		d3.select("#active").text(all.value());
	}
```
1. Also removed line from 129 tp 159 related with cahrts
1. To make python to print \ baclklash charterer
1. based on [this](http://stackoverflow.com/questions/301068/python-backslash-quoting-in-string-literals)
1. Then it is learn that instead of json, meteros web app is taking csv file and getting data from it even the lat and long. These functionalities are included in the drawMap.js and the line ```queue()``` where it referring the csv file is taking care of this data parsing and drawing all the realted process, such as point making in d3 (by lat long parsing) and gving other infor in pop up box.
1. To make the cbe map visualize in d3 canvas, several attempt related with different map, shape file was made. 
1. First attempt was made with recently created ```cbe_webmap.shp``` with all the wards and village was converted into topojson by following command
```bash
ogr2ogr -f GeoJSON cbe_city.json cbe_webmap_city.shp
topojson -o cbe_citytopo.json cbe_city.json
```
1. But this was visulaized in canvas as hapzard rectangualr and scribbled image, for surprise. 
1. So it is doubted the problem with shape file or conversion process. to check this the ```letsmake map``` tutorial referred shape file was converted into toposjon with following command 
```bash
ogr2ogr -f GeoJSON world.json ne_10m_admin_0_map_subunits.shp
topojson -o worldT.json world.json
```
1. This was repolace in the meteros web app topojson and drawmap.js is edited to taking care this change, this was perfectly visulized in the canvas. This indicating the problem is related with shape file. 
1. To ascertain this again a new shape file was created with ne_10m_admin_0_map_subunits.shp opened in qgis. This file named as test_webmapA1.shp was converted into topjson with command 
```
ogr2ogr -f GeoJSON testA1.json test_webmapA1.shp
topojson -o testtopoA1.json testA1.json
```
1. this file was again referen in web app and editing the js file, this also viewed correctly  in the canvas. Thus indicating that the problem is with shape file.
1. So for this, problem might be of wrong projection. Opened the world file and the cbe_webmap.shp and created a new file tracing it. This was ```testtopoA3.json``` made into viewed in the canvas of web app but having problem of zooming.
1. Though changing the values in .scale of var proj and .scaleExtent of var zoom, the zoom is working for single time and then gets strucked.
1. For solving above problem a new shape file as of past test ```test_webmapA2.shp``` wass created and converted into json by 
```bash
ogr2ogr -f GeoJSON testwebA2.json test_webmapA2.shp
topojson -o testtopoA2.json testwebA2.json
```
1. This can be viewed in canvas with zoom and drag functionality. But zoom is still strucking to a maximun limit. This made to know about the functionality of zoom comment in the D3 from [this](https://github.com/mbostock/d3/wiki/Zoom-Behavior). It is found that the command ```.scaleExtent([height*.33, 4 * height])``` is indicating minmum and maximum zooming functioalty. So changing that make more control of map viewing and thus solved the porblem of cbe map view in d3 canvas.
1. This is bascially a simple arthmeatic. To get maximum zoom value, it has to be above the ```.scale(180000)```, so if height is 500, the scale extent has to be 180000/500*height or above
1. With this solution the digitizing cbe map ```testtopoA3.json``` was viewed in the web app and it perfectly gets viewed. Proving onceagain, one day of reading documentation is better than one week of googling. 
1. So the cbe_map overlay with world.shp file was traced with keeping the snapping option to both the shape files. Not giving this 100 pixels units for snapping option eiher of it didn't made snapping option true with turn on enabling topological editing and Enable snapping on intersection.
1. This file was converted into topojson by following command 
```
ogr2ogr -f GeoJSON cbe_webmap.json cbe_webmap_D3A1.shp
topojson -o cbe_webmapT.json cbe_webmap.json
```
1. This was correctly visualized in d3 canvas by giving ```.scale(150000)```,  ```.center(76.9715,11.0142)```, ```.rotate([0,0]);``` and ```.scaleExtent([height*360, 560 * height])```.
1. Further to this the data source, fell.csv file was edited to consider the aerocet data. Before hand the aerocet data was made shorten by removing the unnecessary column by following commands.
```
import os
ad=[]
for file in os.listdir('/home/swl-sacon-dst/Documents/GISE_2013/LAB/Aerocet_DATA/Oct_sample/'):
    if file.endswith("M.csv"):
        ad.append(file)
for file in os.listdir('/home/swl-sacon-dst/Documents/GISE_2013/LAB/Aerocet_DATA/Oct_sample/'):
    if file.endswith("M_N.csv"):
        ad.append(file)
for i in ad:
	db=pd.read_csv(i)
	db1=db[['Time','long','lat','PM2.5(ug/m3)','PM10(ug/m3)']]
	db1.to_csv('web_'+i)
```
1. There was need of minor change in the function ready(),
```.attr("r", 	function(d){return metorScale(d.mass);})```
1. where it takes up the mass_g column of fell.csv for scaling the point marker size. This makes the point data of aeorcet sample completely to be visualized.  
1. Adding the functionality of drop down to choose date
1. This functionality is requiered to visualize the mulitple date Aerocet readings saved in different csv file. For this made a first attempt `webmap_A1` based on [this](http://fiddle.jshell.net/7KJC7/2/) and second attempt `was`webmap_A2` made with [this](http://fiddle.jshell.net/7KJC7/2/) sample code.
1. The attempted functionality was based on drop down select to change the csv file quired in queue function (the lines 51-55 of drawMap.js) are as follows
```python
queue()
	.defer(d3.json, "worldTopo.json")
	.defer(d3.csv, csv)
	.defer(d3.json, "pics.json")
	.await(ready);
```
1. Getting the csv variable value in attempt ```webmap_A1``` by 
```python
var csv = function() {
    var selectedValue = d3.event.target.value;
console.log(selectedValue)}
d3.select("#select-list").on("change", csv);
.defer(d3.csv, selectedValue)
```
1. this attempt was ended with error of no csv file. So made  second attempt ```webmap_A2``` by 
```python
d3.select('#select-list')
  .on('change', function() {
    var csv = eval(d3.select(this).property('value'));
    console.log(csv)
});  
```
1. this attempt was ended with getting of file name with out csv extension based on [this](http://stackoverflow.com/questions/24193593/d3-how-to-change-dataset-based-on-drop-down-box-selection), such as ```web_08102014M.csv```
1. Then it is found that changing csv file is way around method for drop down requirement. There is native d3 method for this requirement named as object constancy. Examples are [this](http://bl.ocks.org/nsonnad/4175202), [this1](https://github.com/mbostock/bost.ocks.org/blob/gh-pages/mike/constancy/index.html) and [this2](http://www.delimited.io/blog/2013/11/8/object-constancy-in-d3)
1. Based on [this](http://bl.ocks.org/nsonnad/4175202) example the csv datafiltering by column/ time is made as follows
```python
rawMetors = csv;
        var nested = d3.nest()
		.key(function(d) { return d.Time; })
		.map(rawMetors)
        var series = menu.property("value");
        // only retrieve data from the selected series, using the nest we just created
        data = nested[series];
        console.log(JSON.stringify(data));
	metors = [];
	data.forEach(function(d){
		d.mass = +d.PM25;
		d.year = +d.year;
		d.id = +d.cartodb_id;
		metors.push(d);
	});
```
1. With the bove code it can filter csv file with time, which is referen cein drop down element
1. So now to create data for the map, stacking all the csv fileinto one csv file by below code
```python
import os
ad=[]
for file in os.listdir('/home/swl-sacon-dst/Documents/GISE_2013/LAB/Aerocet_DATA/Oct_sample/'):
    if file.startswith("web"):
        ad.append(file)
#concatenate pandas dataframe and then convertinginto single csv file
for i in ad:
  db0=pd.DataFrame()
  db=pd.read_csv(i)
  db2=pd.concat([db0, ], axis=0)
	db2.to_csv('web_aerocetdata.csv')
#this is not working, lets try based on [this](http://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-python-pandas-and-concatenate-into-one-dataframe)
frame = pd.DataFrame()
list = []
for files in db:
  df = pd.read_csv(join(directory,files),index_col=None, header=0)
  list.append(df)
frame = pd.concat(list)
#this also not working
#based on [this](http://stackoverflow.com/questions/21149920/pandas-import-multiple-csv-files-into-dataframe-using-a-loop-and-hierarchical-i)
for i in ad
  d = concat([read_csv(f, index_col=0, header=None, axis=1) for f in ad], keys=files)
for i in ad
	d = pd.concat([(pd.read_csv(i, index_col=0, header=0).T) for f in i], keys=i)
#this also not working neds in some error
#
ad1=[]
for i in ad:
  db=pd.read_csv(i)
  db2=df23=pd.concat([Series(row[1], row[0].split(',')) for _, row in db.iterrows()]).reset_index()
  ad1.append(db2)\
#not working
#
names = pd.DataFrame()
for i in ad:
  frame = pd.read_csv(i)
  names = pd.concat(names, frame, ignore_index=True)
#Not working
#
db = pd.DataFrame()
for i in ad:
  frame = pd.read_csv(i)
  db = db.append(frame, ignore_index=True)
#thjis is working
db2=db[['Time','lat','long','PM10(ug/m3)','PM2.5(ug/m3)']]
db3=pd.DataFrame(db2.Time.str.split(" ").tolist())
T1 = pd.merge(db2,db3, on=db2.index, how='outer')
```
1. The 
1. with the above file, a new column was added to give the size of the pointer in the map as circle. The title of the column also has to be changed to plot correctly the map. This makes the points to be plotted in the map but with a problem.
1. The points selected by drop down is not removed and next selection was overlaid with old selction points.
1. For the the change function in BRICS app can be used but problen n queue the json file inside the d3.tranmsion use un the change function.
1. It is found that the above method also is not usable. Basic problem with this is rendering whole objects in canvas that is not economical.
1. Based on [this](http://bl.ocks.org/PBrockmann/230f567762de650953b5) example there is an alternative using the slider and changing/filtering the csv file.
1. For this d3.js slider is basically a object/number genraor in which each slider operation gives new number and so the date in the aerocet data can be gerated by slider by chnageing the date formate to single numerical object refrened with csv file. 
1. To edit the web aerocet file for changing the date formate
```python
import pandas as pd
db=pd.read_csv('web_aerocst.csv')
db['Date2']=pd.to_datetime(db.Date,format='%Y-%m-%d')
db['Date3'] = db['Date2'].map(lambda x: x.strftime('%Y%m%d'))
#to get unique values out of pandas column for dates
db22=pd.Series(db.Date3.values.ravel()).unique()
```
1. This can be referenced in slider as per below
```javascript
//testjsvascript
("#scale").slider({
min: 20141008, // there are 6 days of sampling 0 is all events
max: 20141023,
value: 20141008, //default slider value
step: 1, // step is the allow increments the slider can move. 1 = one day
slide: function(event, ui) {
var months = [0,20141008, 20141021, 20141022, 20141023, 20141017,20141014, 20141020, 20141011];
// handle case where slider at zero = show all sampling
if(ui.value == 0)
{
// highlight all sampling points
d3.selectAll('.samplingpoints')
.style("display","block");
// change the slider text to appropriate date
d3.select("#slidertext")
.text('Attacks During 01/2004 - 12/2009');
}else{
// dim all explosion points
d3.selectAll('.samplingpoints')
.style("display","none");
// get and format current slider value
$tempvalue = ''+months[ui.value]; //look up which month the slider value corresponds to and convert to a string for printing
$tempvalueclass = '._'+$tempvalue; //take month and convert to a class
// highlight all samplingpoints in the selected month by class
d3.selectAll($tempvalueclass)
.style("display","block");
// change the slider text to appropriate date
d3.select("#slidertext")
.text('sampling During '+$tempvalue.substring(4,6)+'/'+tempvalue.substring(0,4));
//}endelse
//}
//});
```
1. Usgae of [this](http://bl.ocks.org/PBrockmann/230f567762de650953b5) example can't be reproduable with the current web app based on metero web app, it seems it is due to different libraries used and projection system used in these two d3.js based web apps. So this attempt named as meteors-master_A5 is left as such due to this lock.
1. A new attempt by adding slider in the native meteors app and console log its working is tried. it would give clues to learn needed functionality. 
1. Changing of libraries in native meteros app for slider is not working. So tried with changing native slider app with libraries of meteros is tried and it is working. So coping meteros app content in ti to make it as a metros app
1. An attempt upto visualization of map, it has ot be defined in svg and other canvas needs of svg. For example if it is a topjson that changes has to be reflected in it blwo code is for topojson visualization by changing a code for json view
```python
var w = 1200;
var h = 500;
var xy = d3.geo.mercator()
          .center([76.9715,11.0100])
		.scale(145000)
		.rotate([0,0]);
var path = d3.geo.path()
    .projection(xy);
var zoom = d3.behavior.zoom()
    .translate(xy.translate())
    .scale(xy.scale())
    .scaleExtent([h*290, 645 * h])
    .on("zoom", zoom);
var svg = d3.select("#graph").append("svg")
		.attr("width", w)
                .attr("height", h)
		.call(zoom);
function zoom() {
	xy.translate(d3.event.translate).scale(d3.event.scale);
	svg.selectAll("path").attr("d", path);
	circles
  		.attr("cx", function(d){return xy([d.long, d.lat])[0];})
		.attr("cy", function(d){return xy([d.long, d.lat])[1];});
}
var borders = svg.append("g");
var states = svg.append("svg:g")
    .attr("id", "states");
var circles = svg.append("svg:g")
    .attr("id", "circles");
var labels = svg.append("svg:g")
    .attr("id", "labels");
d3.json("cbe_webmapT.json", function(error, topology) {
  borders.selectAll("path")
		.data(topojson.object(topology, topology.objects.cbe_webmap)
				.geometries)
	.enter()
		.append("path")
		.attr("d", path)
		.attr("class", "border")
});
```
1. More overt the css also has to be changed, otherwise the the map drawn is filled with black color. The path is important in the case of topojson and it has to be defined in css. 
1. Slide change for data change
1. reflect the slider change in data query of canvas drawing element.  
1. It is very difficult to reuse js scripts already written. Basics of what ;line what behavior it exhibits has to be known to change single line in it. So list what are all required in the app. One try with slider app, another with meteros app
1. with slider app
     1. plot cbe map(it is done in meteors-master_A7 with slider app)
     1. plot the sampling points
     1. give tool tip info of pollution sampled 
     1. change the date with slider or drop down box
1. with meteros app
     1. plot the sampling points(done with meteros app)
     1. give tool tip info of pollution sampled (done with metros app)
     1. change the date with drop down box(it is not working)
1. I think the meteros app can be edited as of slider app, which is more simple and understandable for data filter with slider option. 
1. meteors-master_A9 is doing that- first removing usage of queue library in the app. Then giving the functionality to view the map and get the csv data into it plot it, then have funcotnality of zooming, slider data change and tool tip info view. 
 1. With attempt 1 in the drwamap.js of  meteors-master_A9, it was achived view the map, get the csv data and plot it
 1. with attemp 3, it is achived zooming funcitiality
 1. with attempt 4, achived slider view in the web app, it can't usable with custom dates, alternative is go with drop down box only. The library use for slider is a custom js named d3.slider.js. 
 1. with attempt 5, achived slider functionlity data filtering by slider movment is achived
the decritpiton of all the steps carried ouyt is given in the file of drawMap.js of  meteors-master_A9.
1. The ast attmept gives more insight into the working of d3.js based web app. It basically consist of three compoennts wht we want to achive witht the web app.
 1. view map
 1. plot the sampling point
 1. refresh the sampling point based on the date filter.
the web app contains also threee components for abvove functioalite, all are in main js  file, in current case js file is drawMap.js.  This file conatins three section, one global variable declaaration section, two is file prociessing section which does draw the map(json and csv file), three is the reploting section rteelated with filter tool slider and its associated redraw function. 
1. The filtering carried out in the last attempt has to  be improved to include the normal csv we use it in the apps. The last attempt was carried out using the csv file of slider app and changing lat long to coimbatore. 
1. This attempt will be named as meteors-master_A10 and dealt with drop down funcitronality and redraw funcitoanluty with drop down box using the remaining one and second section as such. 